In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Imports
import numpy as np
import pandas as pd
import os
import sklearn
from  matplotlib import pyplot as plt
from matplotlib.pyplot import subplots
import seaborn as sns

# **EDA**

In [ ]:
#Reading the train meta data
train_data = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/train.csv")
train_data

In [ ]:
# Reading the test metadata
test_data = pd.read_csv("/kaggle/input/petfinder-pawpularity-score/test.csv")
test_data

In [ ]:
#Assigning the X and y columns and performing the train test split
X = train_data.iloc[:,1:13]
X

In [ ]:
y = train_data["Pawpularity"]
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y)

# **Visuals**

In [ ]:
#Checking for correlation between the input and target variable to spot the obvious trends if any
fig, ax = plt.subplots(4, 3, figsize=(15, 15))
for var, subplot in zip(X, ax.flatten()):
    sns.boxplot(x=var, y='Pawpularity', data=train_data, ax=subplot)

Since the boxes of each feature against the target variable does not show a striking difference & the boxes of 0's and 1's are almost at the same level. Hence, there's no direct relationship of each variable with the target.

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(15, 15))
for var, subplot in zip(X, ax.flatten()):
    sns.countplot(x=var, data=train_data, ax=subplot);

From the above histograms, we can deduce that the feature values are highly unbalanced in the given datatset. Hence, before applying the ML models, we need to perform stratified sampling on the train set.

# **Stratified ShuffleSplit cross validator example**

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=77)
sss.get_n_splits(X, y)
print(sss)
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
    y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]

# Linear Regression on the metadata (Baseline)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=77)
sss.get_n_splits(X, y)
# X, y = make_regression(n_samples=9912, n_features=12, random_state=1)
for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
    y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]
    model = LinearRegression()

    model.fit(X_train, y_train)

    importance = model.coef_

    print(importance)

    importance_linear_reg = pd.DataFrame(importance, columns=["Importance"])
    X_columns = train_data.iloc[:,1:13]
    importance_linear_reg["Importance"] = round(importance_linear_reg["Importance"],2)
    importance_linear_reg["Features"] = X_columns.columns
    importance_linear_reg = importance_linear_reg.sort_index(ascending = True, axis = 1)
    importance_linear_reg

    # plot feature importance
    pyplot.bar(importance_linear_reg["Features"], importance_linear_reg["Importance"])
    pyplot.xticks(rotation=90)
    pyplot.show()
    
    y_prediction =  model.predict(X_test)
    y_prediction
    
    print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, y_prediction))
    print("Mean Squared Error:", metrics.mean_squared_error(y_test, y_prediction))
    print("Root Mean Squared Error:", np.sqrt(metrics.mean_absolute_error(y_test, y_prediction)))
    print("Mean Absolute Percentage Error:", np.mean(np.abs((y_test - y_prediction)/y_test))*100)
    
    
    test_pred = model.predict(test_data.iloc[:,1:13])
    print(test_pred)
    
    Solution_Linear_Reg = pd.DataFrame(test_data["Id"], columns=["Id"])
    Solution_Linear_Reg['Pawpularity']=test_pred
    print(Solution_Linear_Reg)

The linear regression gives us a MAE of approximately 15. This means that the prediction can deviate by +-15 pawpularity score. Also, the MAPE is about 78%, hence we need to explore image regression techniques.

# CNN for regression prediction with Keras

In [ ]:
# Preparing for loading the images
from pathlib import Path
import os.path
import pandas as pd

trainpaths = {}
testpaths ={}
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if 'train/' in (os.path.join(dirname, filename)):
            filenames = filename.rstrip(".jpg")
            trainpaths[filenames] = os.path.join(dirname, filename)
        if 'test/' in (os.path.join(dirname, filename)):
            filenames = filename.rstrip(".jpg")
            testpaths[filenames] = os.path.join(dirname, filename)

train_path = pd.DataFrame(list(trainpaths.items()),columns=['Id', 'filepath'])
train_path = train_path.set_index('Id')
train_path
test_path = pd.DataFrame(list(testpaths.items()),columns=['Id', 'filepath'])
test_path= test_path.set_index('Id')
test_path


In [ ]:
#Join the image file paths with the meta data files containing pawpularity 
train_merged = pd.merge(train_data, train_path, on=["Id"], how='left')
train_merged
test_merged = pd.merge(test_data, test_path, on=["Id"], how='left')
test_merged

In [ ]:
#train test split
X = train_merged['filepath']
y = train_merged['Pawpularity']
s = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=77)
s.get_n_splits(X, y)
for train_index, test_index in s.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[list(train_index)], X.iloc[list(test_index)]
    y_train, y_test = y.iloc[list(train_index)], y.iloc[list(test_index)]
train_final = pd.merge(X_train, y_train, left_index=True, right_index=True)
print(train_final)
test_final = pd.merge(X_test, y_test, left_index=True, right_index=True)
print(test_final)

In [ ]:
#Loading the images, rescaling the image pixels from range 0-255 to range 0-1
import tensorflow as tf

train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
rescale =1./255,
validation_split = 0.2
)

test_gen = tf.keras.preprocessing.image.ImageDataGenerator(
rescale = 1./255
)

In [ ]:
#flow from Df function keras: image augmentation

train_images = train_gen.flow_from_dataframe(
    dataframe = train_final,
    x_col = 'filepath',
    y_col = 'Pawpularity',
    target_size = (32,32),
    color_mode = 'rgb',
    shuffle = False,
    class_mode = 'raw',
    subset = 'training',
    batch_size = 32,
    seed = 77    
)

validation_images = train_gen.flow_from_dataframe(
    dataframe = train_final,
    x_col = 'filepath',
    y_col = 'Pawpularity',
    target_size = (32,32),
    color_mode = 'rgb',
    shuffle = False,
    class_mode = 'raw',
    subset = 'validation',
    batch_size = 32,
    seed = 77    
)

test_images = test_gen.flow_from_dataframe(
    dataframe = test_merged,
    x_col = 'filepath',
    y_col = None,
    target_size = (32,32),
    color_mode = 'rgb',
    shuffle = False,
    class_mode = None,
    batch_size = 32,
    seed = 77    
)

In [ ]:
#Training the NN
layer1 = tf.keras.Input(shape=(32,32,3))
layer2 = tf.keras.layers.Conv2D(filters=32, kernel_size = (3,3), padding = 'same')(layer1)
layer3 = tf.keras.activations.relu(layer2, alpha=0.0, max_value=None, threshold=0.0)
layer4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1))(layer3)
layer5 = tf.keras.layers.Conv2D(filters=32, kernel_size = (3,3))(layer4)
layer6 = tf.keras.activations.relu(layer5, alpha=0.0, max_value=None, threshold=0.0)
layer7 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1))(layer6)

layer8 = tf.keras.layers.Conv2D(filters=64, kernel_size = (3,3), padding = 'same')(layer7)
layer9 = tf.keras.activations.relu(layer8, alpha=0.0, max_value=None, threshold=0.0)
layer10 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1))(layer9)
layer11 = tf.keras.layers.Conv2D(filters=64, kernel_size = (3,3))(layer10)
layer12 = tf.keras.activations.relu(layer11, alpha=0.0, max_value=None, threshold=0.0)
layer13 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2),strides=(1, 1))(layer12)

layer14 = tf.keras.layers.GlobalAveragePooling2D()(layer13)

layer15 = tf.keras.layers.Dense(64, activation='relu')(layer14)
layer16 = tf.keras.layers.Dense(64, activation='relu')(layer15)
layer17 = tf.keras.layers.Dense(1, activation='linear')(layer16)

model = tf.keras.Model(inputs = layer1, outputs = layer17)

model.compile(optimizer='adam',loss="mse", metrics=['accuracy'])

modelfit = model.fit(train_images, epochs = 5, validation_data = validation_images,
                    callbacks = [
                        tf.keras.callbacks.EarlyStopping(
                        monitor='val_loss',
                        patience=4,
                        restore_best_weights = True)
                    ])

In [ ]:
#Printing the output of all the intermediate layes to understand the network better
print(layer1)
print(layer2)
print(layer3)
print(layer4)
print(layer5)
print(layer6)
print(layer7)
print(layer8)
print(layer9)
print(layer10)
print(layer11)
print(layer12)
print(layer13)
print(layer14)
print(layer15)
print(layer16)
print(layer17)

In [ ]:
#Train set prediction calculations
import numpy as np
pred = np.squeeze(model.predict(train_images))
print(pred)
real_pawpularity = train_images.labels
print(real_pawpularity)
score = model.evaluate(train_images, verbose=1)
print(score)
np.sqrt(score)

In [ ]:
#Validation set prediction calculations
import numpy as np
pred = np.squeeze(model.predict(validation_images))
print(pred)
real_pawpularity = validation_images.labels
print(real_pawpularity)
score = model.evaluate(validation_images, verbose=1)
print(score)
np.sqrt(score)

In [ ]:
#Test set  prediction calculations
import numpy as np
final_pred = model.predict(test_images)
print(final_pred)

In [ ]:
# Adding the final prediction for the test images into a CSV file
output = pd.DataFrame(test_merged['Id'], columns=["Id"])
output['Pawpularity']=final_pred
output['Pawpularity']= output['Pawpularity'].apply(lambda x: round(x, 2))
output = output.set_index('Id')
print(output)

In [ ]:
output

In [ ]:
# Submit the CSV
output.to_csv('./submission.csv', index = 'Id')